In [1]:
import numpy as np
import pydrake
from pydrake.solvers import ik
from pydrake.examples.pendulum import PendulumWithBotVis

In [2]:
# Construct our model
r = pydrake.rbm.RigidBodyManipulator("../examples/Pendulum/Pendulum.urdf")
lc = pydrake.wrappers.lcm.LCM()
p = PendulumWithBotVis(lc)

In [54]:
# Inverse Kinematics:
# 
# Construct a posture constraint for the pendulum's joint
q = -0.9
posture_constraint = ik.PostureConstraint(r)
posture_constraint.setJointLimits(np.array([[6]], dtype=np.int32), 
                                  np.array([[q]]), 
                                  np.array([[q]]))
# Choose a seed configuration (randomly) and a nominal configuration (at 0)
q_seed = np.vstack((np.zeros((6,1)), 
                    p.getRandomState()[:1]))
q_nom = np.vstack((np.zeros((6,1)), 
                   0.))

# Run the ik solver and draw the result
options = ik.IKoptions(r)
results = pydrake.solvers.ik.inverseKinSimple(r, 
                              q_seed, 
                              q_nom, 
                              [posture_constraint], 
                              options)
p.output(0., np.vstack((results.q_sol[6:], 0.)), np.zeros((1,)))

array([[-0.9],
       [ 0. ]])

In [55]:
# Kinematics and Gradients:
# 
# Pre-compute kinematic information and gradients
r.doKinematics(results.q_sol, np.zeros_like(results.q_sol), True)

In [56]:
# Compute the center of mass and no gradients. The result is a tuple with one element.
r.centerOfMass(0)

(array([[ 0.19583173],
        [ 0.        ],
        [-0.14790249]]),)

In [57]:
# Compute the center of mass and its gradient. The result is a tuple with two elements.
r.centerOfMass(1)

(array([[ 0.19583173],
        [ 0.        ],
        [-0.14790249]]),
 array([[ 1.        ,  0.        ,  0.        ,  0.        , -0.14790249,
          0.        , -0.15540249],
        [ 0.        ,  1.        ,  0.        ,  0.14790249,  0.        ,
          0.19583173,  0.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        , -0.19583173,
          0.        , -0.19583173]]))

In [58]:
# Compute the center of mass and two gradients
r.centerOfMass(2)

(array([[ 0.19583173],
        [ 0.        ],
        [-0.14790249]]),
 array([[ 1.        ,  0.        ,  0.        ,  0.        , -0.14790249,
          0.        , -0.15540249],
        [ 0.        ,  1.        ,  0.        ,  0.14790249,  0.        ,
          0.19583173,  0.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        , -0.19583173,
          0.        , -0.19583173]]),
 array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,